<a href="https://colab.research.google.com/github/oraziotorre/TTinsight/blob/main/TTinsight_DataPreprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TTinsight


---


## Data Preprocessing, Feature Engineering e Model Development

In questo notebook vedremo tutto il processo per il caricamento, l'analisi e la pulizia dei dati. Sceglieremo poi le feature migliori e addestreremo un classificatore binario.

Vogliamo addrestrare un modello che predica la probabilità di "W" nella colonna matcht result

# **Inizializzazione**

In [185]:
import pandas as pd
import numpy as np
from collections import Counter
import ast

# Machine Learning and Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import accuracy_score, roc_auc_score
from keras.preprocessing.sequence import pad_sequences

# Deep Learning (TensorFlow)
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Concatenate, Masking, Embedding, Dropout

from google.colab import data_table

data_table.enable_dataframe_formatter()

In [186]:
dataset = pd.read_csv("raw_dataset.csv")

dataset

,event_id,match_id,match_format,players_gender,match_stage,stage_id,match_duration,match_start_time,player_id,player_2_id,opponent_id,opponent_2_id,player_sets_won,opponent_sets_won,match_scores,sets_required_to_win,current_match_state,player_points,opponent_points
0,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8",3,0-0,"[0, 0, 1, 2, 2, 3, 4, 4, 4, 5, 6, 6, 6, 6, 6, ...","[1, 2, 2, 2, 3, 3, 3, 4, 5, 5, 5, 6, 7, 8, 9, ..."
1,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,117294,109777.0,112463,115009.0,2,3,"11-8,8-11,9-11,11-7,8-11",3,0-0,"[1, 2, 2, 2, 3, 3, 3, 4, 5, 5, 5, 6, 7, 8, 9, ...","[0, 0, 1, 2, 2, 3, 4, 4, 4, 5, 6, 6, 6, 6, 6, ..."
2,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8",3,0-1,"[1, 1, 1, 1, 2, 3, 4, 5, 6, 7, 7, 8, 9, 9, 9, ...","[0, 1, 2, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 5, 6, ..."
3,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,117294,109777.0,112463,115009.0,2,3,"11-8,8-11,9-11,11-7,8-11",3,1-0,"[0, 1, 2, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 5, 6, ...","[1, 1, 1, 1, 2, 3, 4, 5, 6, 7, 7, 8, 9, 9, 9, ..."
4,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8",3,1-1,"[1, 1, 2, 2, 2, 3, 3, 4, 4, 5, 6, 6, 6, 6, 7, ...","[0, 1, 1, 2, 3, 3, 4, 4, 5, 5, 5, 6, 7, 8, 8, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144189,2997,TTEMSINGLES-----------RND2000600----------,S,M,RND2,600,2452,2024-10-29 13:45:00,137238,NaN,132172,NaN,3,2,"11-3,7-11,11-9,7-11,11-6",3,1-1,"[0, 1, 1, 2, 2, 3, 4, 4, 4, 5, 5, 6, 6, 7, 8, ...","[1, 1, 2, 2, 3, 3, 3, 4, 5, 5, 6, 6, 7, 7, 7, ..."
144190,2997,TTEMSINGLES-----------RND2000600----------,S,M,RND2,600,2452,2024-10-29 13:45:00,132172,NaN,137238,NaN,2,3,"3-11,11-7,9-11,11-7,6-11",3,1-2,"[1, 1, 2, 3, 3, 4, 5, 6, 6, 6, 7, 7, 7, 7, 8, ...","[0, 1, 1, 1, 2, 2, 2, 2, 3, 4, 4, 5, 6, 7, 7, ..."
144191,2997,TTEMSINGLES-----------RND2000600----------,S,M,RND2,600,2452,2024-10-29 13:45:00,137238,NaN,132172,NaN,3,2,"11-3,7-11,11-9,7-11,11-6",3,2-1,"[0, 1, 1, 1, 2, 2, 2, 2, 3, 4, 4, 5, 6, 7, 7, ...","[1, 1, 2, 3, 3, 4, 5, 6, 6, 6, 7, 7, 7, 7, 8, ..."
144192,2997,TTEMSINGLES-----------RND2000600----------,S,M,RND2,600,2452,2024-10-29 13:45:00,132172,NaN,137238,NaN,2,3,"3-11,11-7,9-11,11-7,6-11",3,2-2,"[1, 2, 2, 3, 3, 3, 3, 3, 3, 3, 4, 5, 5, 5, 6, 6]","[0, 0, 1, 1, 2, 3, 4, 5, 6, 7, 7, 7, 8, 9, 9, 10]"


# **Etichettatura**

Crea la colonna *set_result* con:


*   **1** per la classe **Vincitori** del set
*   **0** per la classe **Sconfitti** del set



In [187]:
# Funzione per determinare il risultato del set
def calculate_set_result(player_points, opponent_points):

  if isinstance(player_points, str):
    player_points = player_points.strip('[]').split(', ')
    opponent_points = opponent_points.strip('[]').split(', ')

    if int(player_points[-1]) > int(opponent_points[-1]):
        return 1 # Vittoria per il giocatore
    else:
        return 0 # Sconfitta per il giocatore

  else:
    return -1    # Dati non conformi



# Inizializzare una lista vuota per i risultati
set_results = []

# Iterare su ogni riga del DataFrame e calcolare il risultato
for index, row in dataset.iterrows():
    result = calculate_set_result(row['player_points'], row['opponent_points'])
    set_results.append(result)

# Aggiungere la nuova colonna 'set_result' al DataFrame
dataset['set_result'] = set_results

# Visualizzare il risultato
dataset



,event_id,match_id,match_format,players_gender,match_stage,stage_id,match_duration,match_start_time,player_id,player_2_id,opponent_id,opponent_2_id,player_sets_won,opponent_sets_won,match_scores,sets_required_to_win,current_match_state,player_points,opponent_points,set_result
0,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8",3,0-0,"[0, 0, 1, 2, 2, 3, 4, 4, 4, 5, 6, 6, 6, 6, 6, ...","[1, 2, 2, 2, 3, 3, 3, 4, 5, 5, 5, 6, 7, 8, 9, ...",0
1,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,117294,109777.0,112463,115009.0,2,3,"11-8,8-11,9-11,11-7,8-11",3,0-0,"[1, 2, 2, 2, 3, 3, 3, 4, 5, 5, 5, 6, 7, 8, 9, ...","[0, 0, 1, 2, 2, 3, 4, 4, 4, 5, 6, 6, 6, 6, 6, ...",1
2,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8",3,0-1,"[1, 1, 1, 1, 2, 3, 4, 5, 6, 7, 7, 8, 9, 9, 9, ...","[0, 1, 2, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 5, 6, ...",1
3,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,117294,109777.0,112463,115009.0,2,3,"11-8,8-11,9-11,11-7,8-11",3,1-0,"[0, 1, 2, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 5, 6, ...","[1, 1, 1, 1, 2, 3, 4, 5, 6, 7, 7, 8, 9, 9, 9, ...",0
4,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8",3,1-1,"[1, 1, 2, 2, 2, 3, 3, 4, 4, 5, 6, 6, 6, 6, 7, ...","[0, 1, 1, 2, 3, 3, 4, 4, 5, 5, 5, 6, 7, 8, 8, ...",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144189,2997,TTEMSINGLES-----------RND2000600----------,S,M,RND2,600,2452,2024-10-29 13:45:00,137238,NaN,132172,NaN,3,2,"11-3,7-11,11-9,7-11,11-6",3,1-1,"[0, 1, 1, 2, 2, 3, 4, 4, 4, 5, 5, 6, 6, 7, 8, ...","[1, 1, 2, 2, 3, 3, 3, 4, 5, 5, 6, 6, 7, 7, 7, ...",1
144190,2997,TTEMSINGLES-----------RND2000600----------,S,M,RND2,600,2452,2024-10-29 13:45:00,132172,NaN,137238,NaN,2,3,"3-11,11-7,9-11,11-7,6-11",3,1-2,"[1, 1, 2, 3, 3, 4, 5, 6, 6, 6, 7, 7, 7, 7, 8, ...","[0, 1, 1, 1, 2, 2, 2, 2, 3, 4, 4, 5, 6, 7, 7, ...",1
144191,2997,TTEMSINGLES-----------RND2000600----------,S,M,RND2,600,2452,2024-10-29 13:45:00,137238,NaN,132172,NaN,3,2,"11-3,7-11,11-9,7-11,11-6",3,2-1,"[0, 1, 1, 1, 2, 2, 2, 2, 3, 4, 4, 5, 6, 7, 7, ...","[1, 1, 2, 3, 3, 4, 5, 6, 6, 6, 7, 7, 7, 7, 8, ...",0
144192,2997,TTEMSINGLES-----------RND2000600----------,S,M,RND2,600,2452,2024-10-29 13:45:00,132172,NaN,137238,NaN,2,3,"3-11,11-7,9-11,11-7,6-11",3,2-2,"[1, 2, 2, 3, 3, 3, 3, 3, 3, 3, 4, 5, 5, 5, 6, 6]","[0, 0, 1, 1, 2, 3, 4, 5, 6, 7, 7, 7, 8, 9, 9, 10]",0


# **Analisi dei dati**


## Controllo del bilanciamento dei dati
Una delle prime cose che possiamo verificare è se il dataset sia **bilanciato**. Per cui, controlliamo il numero di istanze **1** e **0** della colonna *set_result*

In [188]:

# Numero di elementi per la classe "Vincitori"
print(len(dataset[(dataset['set_result'] == 1)]))
# Numero di elementi per la classe "Sconfitti"
print(len(dataset[(dataset['set_result'] == 0)]))

# Totale
print(len(dataset))


69607
69607
144194


Notiamo che abbiamo un dataset non troppo sbilanciato

## Analisi delle caratteristiche e delle distribuzioni dei dati
Usiamo `describe()` per avere una overview sulle distribuzioni dei dati numerici

In [189]:
# Verifico la correttezza dei campi numerici
dataset.describe()

,event_id,stage_id,match_duration,player_id,player_2_id,opponent_id,opponent_2_id,player_sets_won,opponent_sets_won,sets_required_to_win,set_result
count,144194.000000,144194.000000,144194.000000,144194.000000,35116.000000,144194.000000,35116.000000,144194.000000,144194.000000,144194.000000,144194.000000
mean,2702.487149,1556.146095,2058.918221,123836.818280,125800.822332,123836.818280,125800.822332,2.086800,2.086800,3.146705,0.448195
std,156.933844,5453.961129,1575.253127,17169.115984,17804.203785,17169.115984,17804.203785,1.250522,1.250522,0.353970,0.562487
min,2234.000000,100.000000,0.000000,90051.000000,100001.000000,90051.000000,100001.000000,0.000000,0.000000,2.000000,-1.000000
25%,2569.000000,200.000000,1426.000000,114715.000000,115565.000000,114715.000000,115565.000000,1.000000,1.000000,3.000000,0.000000
50%,2701.000000,600.000000,1914.000000,121403.000000,122428.000000,121403.000000,122428.000000,3.000000,3.000000,3.000000,0.000000
75%,2869.000000,1200.000000,2430.000000,132081.000000,133124.000000,132081.000000,133124.000000,3.000000,3.000000,3.000000,1.000000
max,2997.000000,110005.000000,77074.000000,212088.000000,212090.000000,212088.000000,212090.000000,5.000000,5.000000,5.000000,1.000000


Notiamo che nella colonna "set_required_to_win" sono presenti dei valori rari non nel nostro interesse (2 e 5)

## Controllo dei valori nulli

Verifico i valori mancanti nei dati

In [190]:
nan_mask = dataset.isna()
nan_count = nan_mask.sum()

# Verifico i valori null del raw_dataset
nan_count


,0
event_id,0
match_id,0
match_format,0
players_gender,0
match_stage,0
stage_id,0
match_duration,0
match_start_time,0
player_id,0
player_2_id,109078


#**Operazione sulle feature**


Facciamo alcune considerazioni sulle caratteristiche del nostro dataset e identifichiamo possibili miglioramenti per ottimizzarlo in vista del prossimo training del modello:

* Possibile trasformare i formati T nella colonna *match_format* in formati che siano o S o D

* Le colonne *match_format*, *player_id*, *player_2_id* non sono necessarie per il training del modello e possono essere cancellate

* Le colonne *sets_required_to_win* e *current_match_state* fanno entrambe riferimento al numero di set necessari per vincere una partita. Per rendere il dataset più chiaro e organizzato, possiamo sostituirle con due nuove colonne: *sets_to_win* e *opponent_sets_to_win*, che rappresentano rispettivamente il numero di set necessari al giocatore e all'avversario per vincere la partita.

* La colonna *match_stage* può essere modificata per attribuire maggiore importanza alle partite finali (FNL) rispetto a tutte le altre, migliorando il peso di queste informazioni nel training del modello.

* Per la colonna *set_result* si può semplificare il formato binario e facilitare l'apprendimento del modello.

* Per la colonna *players_gender* si può semplificare il formato tramite l'operazione di *One-Hot Encoding* nel caso di dataset_all_matches o tramite una semplice codifica binaria nel caso del dataset_single_matches

Modifica la colonna *match_format*, sostituendo tutti i valori ' T ' (partita di team) con ' S ' (partita di singolo) o ' D ' (partita di doppio)

In [191]:
def determine_match_format(player_2_id):
    if pd.isna(player_2_id):  # Verifica se player_2_id è NaN (assente)
        return 'S'  # Singolo
    else:
        return 'D'  # Doppio

# Applicare la funzione alla colonna 'match_format' nel dataset
dataset['match_format'] = dataset['player_2_id'].apply(determine_match_format)

# Visualizzare il risultato
dataset

,event_id,match_id,match_format,players_gender,match_stage,stage_id,match_duration,match_start_time,player_id,player_2_id,opponent_id,opponent_2_id,player_sets_won,opponent_sets_won,match_scores,sets_required_to_win,current_match_state,player_points,opponent_points,set_result
0,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8",3,0-0,"[0, 0, 1, 2, 2, 3, 4, 4, 4, 5, 6, 6, 6, 6, 6, ...","[1, 2, 2, 2, 3, 3, 3, 4, 5, 5, 5, 6, 7, 8, 9, ...",0
1,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,117294,109777.0,112463,115009.0,2,3,"11-8,8-11,9-11,11-7,8-11",3,0-0,"[1, 2, 2, 2, 3, 3, 3, 4, 5, 5, 5, 6, 7, 8, 9, ...","[0, 0, 1, 2, 2, 3, 4, 4, 4, 5, 6, 6, 6, 6, 6, ...",1
2,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8",3,0-1,"[1, 1, 1, 1, 2, 3, 4, 5, 6, 7, 7, 8, 9, 9, 9, ...","[0, 1, 2, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 5, 6, ...",1
3,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,117294,109777.0,112463,115009.0,2,3,"11-8,8-11,9-11,11-7,8-11",3,1-0,"[0, 1, 2, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 5, 6, ...","[1, 1, 1, 1, 2, 3, 4, 5, 6, 7, 7, 8, 9, 9, 9, ...",0
4,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8",3,1-1,"[1, 1, 2, 2, 2, 3, 3, 4, 4, 5, 6, 6, 6, 6, 7, ...","[0, 1, 1, 2, 3, 3, 4, 4, 5, 5, 5, 6, 7, 8, 8, ...",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144189,2997,TTEMSINGLES-----------RND2000600----------,S,M,RND2,600,2452,2024-10-29 13:45:00,137238,NaN,132172,NaN,3,2,"11-3,7-11,11-9,7-11,11-6",3,1-1,"[0, 1, 1, 2, 2, 3, 4, 4, 4, 5, 5, 6, 6, 7, 8, ...","[1, 1, 2, 2, 3, 3, 3, 4, 5, 5, 6, 6, 7, 7, 7, ...",1
144190,2997,TTEMSINGLES-----------RND2000600----------,S,M,RND2,600,2452,2024-10-29 13:45:00,132172,NaN,137238,NaN,2,3,"3-11,11-7,9-11,11-7,6-11",3,1-2,"[1, 1, 2, 3, 3, 4, 5, 6, 6, 6, 7, 7, 7, 7, 8, ...","[0, 1, 1, 1, 2, 2, 2, 2, 3, 4, 4, 5, 6, 7, 7, ...",1
144191,2997,TTEMSINGLES-----------RND2000600----------,S,M,RND2,600,2452,2024-10-29 13:45:00,137238,NaN,132172,NaN,3,2,"11-3,7-11,11-9,7-11,11-6",3,2-1,"[0, 1, 1, 1, 2, 2, 2, 2, 3, 4, 4, 5, 6, 7, 7, ...","[1, 1, 2, 3, 3, 4, 5, 6, 6, 6, 7, 7, 7, 7, 8, ...",0
144192,2997,TTEMSINGLES-----------RND2000600----------,S,M,RND2,600,2452,2024-10-29 13:45:00,132172,NaN,137238,NaN,2,3,"3-11,11-7,9-11,11-7,6-11",3,2-2,"[1, 2, 2, 3, 3, 3, 3, 3, 3, 3, 4, 5, 5, 5, 6, 6]","[0, 0, 1, 1, 2, 3, 4, 5, 6, 7, 7, 7, 8, 9, 9, 10]",0


In base a ciò che abbiamo appreso precedentemente da `describe()`, rimuoviamo i valori non desiderati della colonna *sets_required_to_win*

In [192]:
# Rimuovo le righe contenenti set con punteggi rari che non voglio considerare
dataset = dataset[~dataset['sets_required_to_win'].isin([2, 5])]

dataset.describe()

,event_id,stage_id,match_duration,player_id,player_2_id,opponent_id,opponent_2_id,player_sets_won,opponent_sets_won,sets_required_to_win,set_result
count,144186.000000,144186.000000,144186.000000,144186.000000,35116.000000,144186.000000,35116.000000,144186.000000,144186.000000,144186.000000,144186.000000
mean,2702.497746,1556.146436,2058.904013,123837.064202,125800.822332,123837.064202,125800.822332,2.086791,2.086791,3.146686,0.448213
std,156.931733,5454.110892,1575.282189,17169.454336,17804.203785,17169.454336,17804.203785,1.250485,1.250485,0.353793,0.562468
min,2234.000000,100.000000,0.000000,90051.000000,100001.000000,90051.000000,100001.000000,0.000000,0.000000,3.000000,-1.000000
25%,2569.000000,200.000000,1426.000000,114715.000000,115565.000000,114715.000000,115565.000000,1.000000,1.000000,3.000000,0.000000
50%,2701.000000,600.000000,1914.000000,121403.000000,122428.000000,121403.000000,122428.000000,3.000000,3.000000,3.000000,0.000000
75%,2869.000000,1200.000000,2430.000000,132081.000000,133124.000000,132081.000000,133124.000000,3.000000,3.000000,3.000000,1.000000
max,2997.000000,110005.000000,77074.000000,212088.000000,212090.000000,212088.000000,212090.000000,4.000000,4.000000,4.000000,1.000000


Eliminiamo i valori nulli della colonna *player points*

In [193]:
# Tolgo tutti i set con punteggi nulli
dataset = dataset.dropna(subset=["player_points"])

dataset[dataset['player_points'].isnull()]

,event_id,match_id,match_format,players_gender,match_stage,stage_id,match_duration,match_start_time,player_id,player_2_id,opponent_id,opponent_2_id,player_sets_won,opponent_sets_won,match_scores,sets_required_to_win,current_match_state,player_points,opponent_points,set_result


##**Colonna *match_stage***

Al posto della colonna *match_stage* inseriamo *is_final* in modo da semplificarne i valori in:
- **FNL = 1**
- **Tutti gli altri stage del torneo = 0**

In questo modo possiamo dare maggiore importanza alle partite finali (FNL) rispetto a tutte le altre, **migliorando il peso di queste informazioni** nel training del modello.

In [194]:
# Creiamo la colonna 'is_final' il cui valore è 1 se 'match_stage' == 'FNL', altrimenti 0
dataset['is_final'] = (dataset['match_stage'] == 'FNL').astype(int)

# Rimuoviamo la colonna 'match_stage'
dataset = dataset.drop(columns=['match_stage'])

players_gender_index = dataset.columns.get_loc('players_gender')
is_final_column = dataset.pop('is_final')
dataset.insert(players_gender_index + 1, 'is_final',  is_final_column)

dataset

<ipython-input-194-3aee278e3ccb>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['is_final'] = (dataset['match_stage'] == 'FNL').astype(int)


,event_id,match_id,match_format,players_gender,is_final,stage_id,match_duration,match_start_time,player_id,player_2_id,opponent_id,opponent_2_id,player_sets_won,opponent_sets_won,match_scores,sets_required_to_win,current_match_state,player_points,opponent_points,set_result
0,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,1,100,1990,2021-11-15 17:32:48,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8",3,0-0,"[0, 0, 1, 2, 2, 3, 4, 4, 4, 5, 6, 6, 6, 6, 6, ...","[1, 2, 2, 2, 3, 3, 3, 4, 5, 5, 5, 6, 7, 8, 9, ...",0
1,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,1,100,1990,2021-11-15 17:32:48,117294,109777.0,112463,115009.0,2,3,"11-8,8-11,9-11,11-7,8-11",3,0-0,"[1, 2, 2, 2, 3, 3, 3, 4, 5, 5, 5, 6, 7, 8, 9, ...","[0, 0, 1, 2, 2, 3, 4, 4, 4, 5, 6, 6, 6, 6, 6, ...",1
2,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,1,100,1990,2021-11-15 17:32:48,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8",3,0-1,"[1, 1, 1, 1, 2, 3, 4, 5, 6, 7, 7, 8, 9, 9, 9, ...","[0, 1, 2, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 5, 6, ...",1
3,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,1,100,1990,2021-11-15 17:32:48,117294,109777.0,112463,115009.0,2,3,"11-8,8-11,9-11,11-7,8-11",3,1-0,"[0, 1, 2, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 5, 6, ...","[1, 1, 1, 1, 2, 3, 4, 5, 6, 7, 7, 8, 9, 9, 9, ...",0
4,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,1,100,1990,2021-11-15 17:32:48,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8",3,1-1,"[1, 1, 2, 2, 2, 3, 3, 4, 4, 5, 6, 6, 6, 6, 7, ...","[0, 1, 1, 2, 3, 3, 4, 4, 5, 5, 5, 6, 7, 8, 8, ...",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144189,2997,TTEMSINGLES-----------RND2000600----------,S,M,0,600,2452,2024-10-29 13:45:00,137238,NaN,132172,NaN,3,2,"11-3,7-11,11-9,7-11,11-6",3,1-1,"[0, 1, 1, 2, 2, 3, 4, 4, 4, 5, 5, 6, 6, 7, 8, ...","[1, 1, 2, 2, 3, 3, 3, 4, 5, 5, 6, 6, 7, 7, 7, ...",1
144190,2997,TTEMSINGLES-----------RND2000600----------,S,M,0,600,2452,2024-10-29 13:45:00,132172,NaN,137238,NaN,2,3,"3-11,11-7,9-11,11-7,6-11",3,1-2,"[1, 1, 2, 3, 3, 4, 5, 6, 6, 6, 7, 7, 7, 7, 8, ...","[0, 1, 1, 1, 2, 2, 2, 2, 3, 4, 4, 5, 6, 7, 7, ...",1
144191,2997,TTEMSINGLES-----------RND2000600----------,S,M,0,600,2452,2024-10-29 13:45:00,137238,NaN,132172,NaN,3,2,"11-3,7-11,11-9,7-11,11-6",3,2-1,"[0, 1, 1, 1, 2, 2, 2, 2, 3, 4, 4, 5, 6, 7, 7, ...","[1, 1, 2, 3, 3, 4, 5, 6, 6, 6, 7, 7, 7, 7, 8, ...",0
144192,2997,TTEMSINGLES-----------RND2000600----------,S,M,0,600,2452,2024-10-29 13:45:00,132172,NaN,137238,NaN,2,3,"3-11,11-7,9-11,11-7,6-11",3,2-2,"[1, 2, 2, 3, 3, 3, 3, 3, 3, 3, 4, 5, 5, 5, 6, 6]","[0, 0, 1, 1, 2, 3, 4, 5, 6, 7, 7, 7, 8, 9, 9, 10]",0


##**Colonne *sets_required_to_win* e *current_match_state***

Da queste due colonne prendiamo le informazioni più importanti e creiamo due nuove colonne

In [195]:
final_set_diff = dataset['sets_required_to_win'] - dataset['current_match_state'].str.split('-', expand=True)[0].astype(int)
opponent_final_set_diff = dataset['sets_required_to_win'] - dataset['current_match_state'].str.split('-', expand=True)[1].astype(int)

# Assegna 1 se il risultato è 1, altrimenti 0
dataset['final_set'] = (final_set_diff == 1).astype(int)
dataset['opponent_final_set'] = (opponent_final_set_diff == 1).astype(int)

is_final_index = dataset.columns.get_loc('is_final')
final_set_column = dataset.pop('final_set')
opponent_final_set_column = dataset.pop('opponent_final_set')
dataset.insert(is_final_index + 1, 'final_set',  final_set_column)
dataset.insert(is_final_index + 2, 'opponent_final_set',opponent_final_set_column)


# Elimina le colonne originali
dataset.drop(['sets_required_to_win', 'current_match_state'], axis=1, inplace=True)

dataset

,event_id,match_id,match_format,players_gender,is_final,final_set,opponent_final_set,stage_id,match_duration,match_start_time,player_id,player_2_id,opponent_id,opponent_2_id,player_sets_won,opponent_sets_won,match_scores,player_points,opponent_points,set_result
0,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,1,0,0,100,1990,2021-11-15 17:32:48,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8","[0, 0, 1, 2, 2, 3, 4, 4, 4, 5, 6, 6, 6, 6, 6, ...","[1, 2, 2, 2, 3, 3, 3, 4, 5, 5, 5, 6, 7, 8, 9, ...",0
1,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,1,0,0,100,1990,2021-11-15 17:32:48,117294,109777.0,112463,115009.0,2,3,"11-8,8-11,9-11,11-7,8-11","[1, 2, 2, 2, 3, 3, 3, 4, 5, 5, 5, 6, 7, 8, 9, ...","[0, 0, 1, 2, 2, 3, 4, 4, 4, 5, 6, 6, 6, 6, 6, ...",1
2,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,1,0,0,100,1990,2021-11-15 17:32:48,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8","[1, 1, 1, 1, 2, 3, 4, 5, 6, 7, 7, 8, 9, 9, 9, ...","[0, 1, 2, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 5, 6, ...",1
3,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,1,0,0,100,1990,2021-11-15 17:32:48,117294,109777.0,112463,115009.0,2,3,"11-8,8-11,9-11,11-7,8-11","[0, 1, 2, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 5, 6, ...","[1, 1, 1, 1, 2, 3, 4, 5, 6, 7, 7, 8, 9, 9, 9, ...",0
4,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,1,0,0,100,1990,2021-11-15 17:32:48,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8","[1, 1, 2, 2, 2, 3, 3, 4, 4, 5, 6, 6, 6, 6, 7, ...","[0, 1, 1, 2, 3, 3, 4, 4, 5, 5, 5, 6, 7, 8, 8, ...",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144189,2997,TTEMSINGLES-----------RND2000600----------,S,M,0,0,0,600,2452,2024-10-29 13:45:00,137238,NaN,132172,NaN,3,2,"11-3,7-11,11-9,7-11,11-6","[0, 1, 1, 2, 2, 3, 4, 4, 4, 5, 5, 6, 6, 7, 8, ...","[1, 1, 2, 2, 3, 3, 3, 4, 5, 5, 6, 6, 7, 7, 7, ...",1
144190,2997,TTEMSINGLES-----------RND2000600----------,S,M,0,0,1,600,2452,2024-10-29 13:45:00,132172,NaN,137238,NaN,2,3,"3-11,11-7,9-11,11-7,6-11","[1, 1, 2, 3, 3, 4, 5, 6, 6, 6, 7, 7, 7, 7, 8, ...","[0, 1, 1, 1, 2, 2, 2, 2, 3, 4, 4, 5, 6, 7, 7, ...",1
144191,2997,TTEMSINGLES-----------RND2000600----------,S,M,0,1,0,600,2452,2024-10-29 13:45:00,137238,NaN,132172,NaN,3,2,"11-3,7-11,11-9,7-11,11-6","[0, 1, 1, 1, 2, 2, 2, 2, 3, 4, 4, 5, 6, 7, 7, ...","[1, 1, 2, 3, 3, 4, 5, 6, 6, 6, 7, 7, 7, 7, 8, ...",0
144192,2997,TTEMSINGLES-----------RND2000600----------,S,M,0,1,1,600,2452,2024-10-29 13:45:00,132172,NaN,137238,NaN,2,3,"3-11,11-7,9-11,11-7,6-11","[1, 2, 2, 3, 3, 3, 3, 3, 3, 3, 4, 5, 5, 5, 6, 6]","[0, 0, 1, 1, 2, 3, 4, 5, 6, 7, 7, 7, 8, 9, 9, 10]",0


## Colonna *players_gender*

Questa colonna verrà utilizzata per il **one-hot encoding**, che trasformerà le seguenti combinazioni di genere in vettori:

- **M** → `[1, 0, 0]` (Maschile)
- **W** → `[0, 1, 0]` (Femminile)
- **X** → `[0, 0, 1]` (Non specificato)

In [196]:
encoder = OneHotEncoder(sparse_output=False)  # sparse_output=False per ottenere una matrice densa

# Trasformiamo i dati facendo One Hot Encoding sulla colonna players_gender
X_encoded = encoder.fit_transform(dataset[['players_gender']])

# Otteniamo i nomi delle nuove colonne
encoded_columns = encoder.get_feature_names_out(['players_gender'])

dataset[encoded_columns] = X_encoded
dataset[encoded_columns] = dataset[encoded_columns].astype(int)

# Rimuoviamo la colonna 'players_gender'
dataset = dataset.drop(columns=['players_gender'])

# Spostiamo le colonne codificate all'inizio
encoded_columns = ['players_gender_M', 'players_gender_W', 'players_gender_X']
dataset = dataset[encoded_columns + [col for col in dataset.columns if col not in encoded_columns]]

# Visualizziamo il risultato
dataset


,players_gender_M,players_gender_W,players_gender_X,event_id,match_id,match_format,is_final,final_set,opponent_final_set,stage_id,...,player_id,player_2_id,opponent_id,opponent_2_id,player_sets_won,opponent_sets_won,match_scores,player_points,opponent_points,set_result
0,0,1,0,2234,TTEWDOUBLES-----------FNL-000100----------,D,1,0,0,100,...,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8","[0, 0, 1, 2, 2, 3, 4, 4, 4, 5, 6, 6, 6, 6, 6, ...","[1, 2, 2, 2, 3, 3, 3, 4, 5, 5, 5, 6, 7, 8, 9, ...",0
1,0,1,0,2234,TTEWDOUBLES-----------FNL-000100----------,D,1,0,0,100,...,117294,109777.0,112463,115009.0,2,3,"11-8,8-11,9-11,11-7,8-11","[1, 2, 2, 2, 3, 3, 3, 4, 5, 5, 5, 6, 7, 8, 9, ...","[0, 0, 1, 2, 2, 3, 4, 4, 4, 5, 6, 6, 6, 6, 6, ...",1
2,0,1,0,2234,TTEWDOUBLES-----------FNL-000100----------,D,1,0,0,100,...,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8","[1, 1, 1, 1, 2, 3, 4, 5, 6, 7, 7, 8, 9, 9, 9, ...","[0, 1, 2, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 5, 6, ...",1
3,0,1,0,2234,TTEWDOUBLES-----------FNL-000100----------,D,1,0,0,100,...,117294,109777.0,112463,115009.0,2,3,"11-8,8-11,9-11,11-7,8-11","[0, 1, 2, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 5, 6, ...","[1, 1, 1, 1, 2, 3, 4, 5, 6, 7, 7, 8, 9, 9, 9, ...",0
4,0,1,0,2234,TTEWDOUBLES-----------FNL-000100----------,D,1,0,0,100,...,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8","[1, 1, 2, 2, 2, 3, 3, 4, 4, 5, 6, 6, 6, 6, 7, ...","[0, 1, 1, 2, 3, 3, 4, 4, 5, 5, 5, 6, 7, 8, 8, ...",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144189,1,0,0,2997,TTEMSINGLES-----------RND2000600----------,S,0,0,0,600,...,137238,NaN,132172,NaN,3,2,"11-3,7-11,11-9,7-11,11-6","[0, 1, 1, 2, 2, 3, 4, 4, 4, 5, 5, 6, 6, 7, 8, ...","[1, 1, 2, 2, 3, 3, 3, 4, 5, 5, 6, 6, 7, 7, 7, ...",1
144190,1,0,0,2997,TTEMSINGLES-----------RND2000600----------,S,0,0,1,600,...,132172,NaN,137238,NaN,2,3,"3-11,11-7,9-11,11-7,6-11","[1, 1, 2, 3, 3, 4, 5, 6, 6, 6, 7, 7, 7, 7, 8, ...","[0, 1, 1, 1, 2, 2, 2, 2, 3, 4, 4, 5, 6, 7, 7, ...",1
144191,1,0,0,2997,TTEMSINGLES-----------RND2000600----------,S,0,1,0,600,...,137238,NaN,132172,NaN,3,2,"11-3,7-11,11-9,7-11,11-6","[0, 1, 1, 1, 2, 2, 2, 2, 3, 4, 4, 5, 6, 7, 7, ...","[1, 1, 2, 3, 3, 4, 5, 6, 6, 6, 7, 7, 7, 7, 8, ...",0
144192,1,0,0,2997,TTEMSINGLES-----------RND2000600----------,S,0,1,1,600,...,132172,NaN,137238,NaN,2,3,"3-11,11-7,9-11,11-7,6-11","[1, 2, 2, 3, 3, 3, 3, 3, 3, 3, 4, 5, 5, 5, 6, 6]","[0, 0, 1, 1, 2, 3, 4, 5, 6, 7, 7, 7, 8, 9, 9, 10]",0



##**Operazione su *single_matches_dataset***

Facciamo adesso alcune operazioni specifiche sulle feature su un dataset composto unicamente da partite di singolo


In [197]:
# Rimuovo dal dataset le partite di doppio
rows_to_remove = dataset[dataset['match_format'] == 'D'].index

# Rimuovi le righe corrispondenti dal dataset originale
dataset_singles = dataset.drop(index=rows_to_remove)

# Rimuovo le colonne non desiderate poichè "players_gender_X" non può essere mai 1 perchè non ci sono doppi misti
columns_to_drop = ["players_gender_W", "players_gender_X"]
dataset_singles = dataset_singles.drop(columns=columns_to_drop, axis=1)

# Modifico il nome della tabella in "players_gender", 'M' = 1 e 'W' = 0
dataset_singles.rename(columns={"players_gender_M": "players_gender"}, inplace=True)

dataset_singles

,players_gender,event_id,match_id,match_format,is_final,final_set,opponent_final_set,stage_id,match_duration,match_start_time,player_id,player_2_id,opponent_id,opponent_2_id,player_sets_won,opponent_sets_won,match_scores,player_points,opponent_points,set_result
16,1,2234,TTEMTEAM--------------FNL-00010001--------,S,1,0,0,10001,2387,2021-11-19 23:25:33,111683,NaN,113419,NaN,3,2,"11-8,11-7,7-11,7-11,11-5","[0, 1, 2, 2, 2, 3, 3, 4, 4, 5, 6, 6, 6, 7, 7, ...","[1, 1, 1, 2, 3, 3, 4, 4, 5, 5, 5, 6, 7, 7, 8, ...",1
17,1,2234,TTEMTEAM--------------FNL-00010001--------,S,1,0,0,10001,2387,2021-11-19 23:25:33,113419,NaN,111683,NaN,2,3,"8-11,7-11,11-7,11-7,5-11","[1, 1, 1, 2, 3, 3, 4, 4, 5, 5, 5, 6, 7, 7, 8, ...","[0, 1, 2, 2, 2, 3, 3, 4, 4, 5, 6, 6, 6, 7, 7, ...",0
18,1,2234,TTEMTEAM--------------FNL-00010001--------,S,1,0,0,10001,2387,2021-11-19 23:25:33,111683,NaN,113419,NaN,3,2,"11-8,11-7,7-11,7-11,11-5","[0, 0, 1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 9, 10, 10...","[1, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 5, ...",1
19,1,2234,TTEMTEAM--------------FNL-00010001--------,S,1,0,0,10001,2387,2021-11-19 23:25:33,113419,NaN,111683,NaN,2,3,"8-11,7-11,11-7,11-7,5-11","[1, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 5, ...","[0, 0, 1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 9, 10, 10...",0
20,1,2234,TTEMTEAM--------------FNL-00010001--------,S,1,1,0,10001,2387,2021-11-19 23:25:33,111683,NaN,113419,NaN,3,2,"11-8,11-7,7-11,7-11,11-5","[1, 1, 2, 2, 3, 4, 4, 5, 5, 6, 6, 6, 6, 7, 7, ...","[0, 1, 1, 2, 2, 2, 3, 3, 4, 4, 5, 6, 7, 7, 8, ...",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144189,1,2997,TTEMSINGLES-----------RND2000600----------,S,0,0,0,600,2452,2024-10-29 13:45:00,137238,NaN,132172,NaN,3,2,"11-3,7-11,11-9,7-11,11-6","[0, 1, 1, 2, 2, 3, 4, 4, 4, 5, 5, 6, 6, 7, 8, ...","[1, 1, 2, 2, 3, 3, 3, 4, 5, 5, 6, 6, 7, 7, 7, ...",1
144190,1,2997,TTEMSINGLES-----------RND2000600----------,S,0,0,1,600,2452,2024-10-29 13:45:00,132172,NaN,137238,NaN,2,3,"3-11,11-7,9-11,11-7,6-11","[1, 1, 2, 3, 3, 4, 5, 6, 6, 6, 7, 7, 7, 7, 8, ...","[0, 1, 1, 1, 2, 2, 2, 2, 3, 4, 4, 5, 6, 7, 7, ...",1
144191,1,2997,TTEMSINGLES-----------RND2000600----------,S,0,1,0,600,2452,2024-10-29 13:45:00,137238,NaN,132172,NaN,3,2,"11-3,7-11,11-9,7-11,11-6","[0, 1, 1, 1, 2, 2, 2, 2, 3, 4, 4, 5, 6, 7, 7, ...","[1, 1, 2, 3, 3, 4, 5, 6, 6, 6, 7, 7, 7, 7, 8, ...",0
144192,1,2997,TTEMSINGLES-----------RND2000600----------,S,0,1,1,600,2452,2024-10-29 13:45:00,132172,NaN,137238,NaN,2,3,"3-11,11-7,9-11,11-7,6-11","[1, 2, 2, 3, 3, 3, 3, 3, 3, 3, 4, 5, 5, 5, 6, 6]","[0, 0, 1, 1, 2, 3, 4, 5, 6, 7, 7, 7, 8, 9, 9, 10]",0


## Eliminazione delle colonne non adatte all'addestramento

In [198]:
columns_to_drop = ['event_id', 'match_id', 'stage_id', 'match_duration',
                  'match_start_time','opponent_id', 'opponent_2_id',
                  'player_sets_won', 'opponent_sets_won', 'match_scores',
                  'opponent_points','match_format', 'player_id', 'player_2_id']

# Rimuovo le colonne che non mi interessano
dataset = dataset.drop(columns=columns_to_drop, axis=1)
dataset_singles = dataset_singles.drop(columns=columns_to_drop, axis=1)

dataset_singles

,players_gender,is_final,final_set,opponent_final_set,player_points,set_result
16,1,1,0,0,"[0, 1, 2, 2, 2, 3, 3, 4, 4, 5, 6, 6, 6, 7, 7, ...",1
17,1,1,0,0,"[1, 1, 1, 2, 3, 3, 4, 4, 5, 5, 5, 6, 7, 7, 8, ...",0
18,1,1,0,0,"[0, 0, 1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 9, 10, 10...",1
19,1,1,0,0,"[1, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 5, ...",0
20,1,1,1,0,"[1, 1, 2, 2, 3, 4, 4, 5, 5, 6, 6, 6, 6, 7, 7, ...",0
...,...,...,...,...,...,...
144189,1,0,0,0,"[0, 1, 1, 2, 2, 3, 4, 4, 4, 5, 5, 6, 6, 7, 8, ...",1
144190,1,0,0,1,"[1, 1, 2, 3, 3, 4, 5, 6, 6, 6, 7, 7, 7, 7, 8, ...",1
144191,1,0,1,0,"[0, 1, 1, 1, 2, 2, 2, 2, 3, 4, 4, 5, 6, 7, 7, ...",0
144192,1,0,1,1,"[1, 2, 2, 3, 3, 3, 3, 3, 3, 3, 4, 5, 5, 5, 6, 6]",0


Salvataggio dei nuovi dataset

In [199]:
#Salvo il dataset pulito dai valori indesiderati
dataset.to_csv("cleaned_dataset.csv", index=False)

In [200]:
# Salvo il dataset finale delle partite di singolo
dataset_singles.to_csv("singles_matches_dataset.csv", index=False)

#**Modello**
Dobbiamo considerare tutte le istanze di un singolo punteggio per poter allenare il modello
Quindi per ciascuna istanza eseguiamo la seguente funzione(questo per la regressione logistica e per il testing dell'LSTM):

In [201]:
'''
#Funzione per generare la sequenza dei punteggi
def generate_score_sequence(player_points):
    points_list = player_points.strip('[]').split(', ')  # Pulizia e separazione dei punteggi
    sequence = [list(map(int, points_list[:i])) for i in range(1, len(points_list) + 1)]  # Liste di interi
    return sequence

#Generiamo e esplodiamo la colonna
dataset_singles['player_points'] = dataset_singles['player_points'].apply(generate_score_sequence)
dataset_singles = dataset_singles.explode('player_points', ignore_index=True)

#Mostriamo il risultato finale
dataset_singles
'''

,players_gender,is_final,final_set,opponent_final_set,player_points,set_result
0,1,1,0,0,"[0, 1, 2, 2, 2, 3, 3, 4, 4, 5, 6, 6, 6, 7, 7, ...",1
1,1,1,0,0,"[1, 1, 1, 2, 3, 3, 4, 4, 5, 5, 5, 6, 7, 7, 8, ...",0
2,1,1,0,0,"[0, 0, 1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 9, 10, 10...",1
3,1,1,0,0,"[1, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 5, ...",0
4,1,1,1,0,"[1, 1, 2, 2, 3, 4, 4, 5, 5, 6, 6, 6, 6, 7, 7, ...",0
...,...,...,...,...,...,...
105449,1,0,0,0,"[0, 1, 1, 2, 2, 3, 4, 4, 4, 5, 5, 6, 6, 7, 8, ...",1
105450,1,0,0,1,"[1, 1, 2, 3, 3, 4, 5, 6, 6, 6, 7, 7, 7, 7, 8, ...",1
105451,1,0,1,0,"[0, 1, 1, 1, 2, 2, 2, 2, 3, 4, 4, 5, 6, 7, 7, ...",0
105452,1,0,1,1,"[1, 2, 2, 3, 3, 3, 3, 3, 3, 3, 4, 5, 5, 5, 6, 6]",0


In [202]:
sequence_lengths = dataset_singles['player_points'].apply(len)

length_counts = sequence_lengths.value_counts().sort_index()

print(length_counts)

player_points
10      166
11     1276
12     2914
13     5372
14     8214
15    11198
16    13592
17    14990
18    16038
19    16086
21     7720
23     3946
25     1948
27      952
29      558
31      238
33       98
35       80
37       28
39       22
41       12
43        2
45        4
Name: count, dtype: int64


In [203]:
padded_sequences = pad_sequences(
    dataset_singles['player_points'],
    maxlen=19,
    padding='post',
    truncating='post',
    value=-1
)
print(padded_sequences.shape)

(105454, 19)


Adesso applichiamo il dataset finale a cui è stato applicato padding. **Attenzione il padding quando fai LSTM non deve essere sul dataset ingigantito

Utilizziamo un modello di regressione logistica

In [204]:
#X_bin = np.array(dataset_singles[['players_gender', 'is_final', 'final_set', 'opponent_final_set']].values)  # Concatenazione feature
X = np.array(padded_sequences)

#X_concate = [np.concatenate([x_bin, seq]) for x_bin, seq in zip(X_bin, X_seq)]
#X = np.array(X_concate)

Y = np.array(dataset_singles['set_result'].values)  # Target binari

In [205]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [217]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
model = LogisticRegression(max_iter=1000, random_state=50)

# Addestramento
model.fit(X_train_scaled, Y_train)

# Predizioni
Y_pred = model.predict(X_test_scaled)
Y_pred_prob = model.predict_proba(X_test_scaled)

# Valutazione
accuracy = accuracy_score(Y_test, Y_pred)
print(f"Accuracy: {accuracy:.2f}")
print(classification_report(Y_test, Y_pred))


Accuracy: 0.80
              precision    recall  f1-score   support

           0       0.81      0.79      0.80     10529
           1       0.80      0.81      0.80     10562

    accuracy                           0.80     21091
   macro avg       0.80      0.80      0.80     21091
weighted avg       0.80      0.80      0.80     21091



In [220]:
'''
Per adesso ho notato che il modello predice bene i casi con
score positi e vicini alla vittoria ma vacilla con score
negativi e vicini alla sconfitta.


In generale il problema potrebbe essere dato dalla mancata generalizzazione
delle troppe feature associato a un numero troppo elevato di istanze
con troppi valori di padding

Infatti sembra che predice meglio i casi semplici

Effettuando il padding su tutti gli elementi abbiamo notato che il modello




'''

test_bin = np.array([[1, 0, 1, 0], [1, 0, 1, 0]])  # [players_gender, is_final, final_set, opponent_final_set]
test_seq = np.array([[1, 2, 3, 4, 5, 6, 7, 8, 9, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
                     [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, -1, -1, -1, -1, -1, -1, -1, -1]])  # Punti del giocatore

# Concatenare le due parti per formare il caso di test finale
#X_test_case_concate = [np.concatenate([x_bin, seq]) for x_bin, seq in zip(test_bin, test_seq)]
#X_test_case = np.array(X_test_case_concate)

X_test_case = scaler.transform(test_seq)

probabilities = model.predict_proba([X_test_case[1]])

# La probabilità per la classe 1 è nella seconda colonna (indice 1)
prob_class_1 = probabilities[0][1]

# La probabilità per la classe 0 è nella prima colonna (indice 0)
prob_class_0 = probabilities[0][0]

# Mostra il risultato
print(f"Probabilità di vincere (classe 1): {prob_class_1:.4f}")
print(f"Probabilità di perdere (classe 0): {prob_class_0:.4f}")

Probabilità di vincere (classe 1): 0.0004
Probabilità di perdere (classe 0): 0.9996


LSTM

Effettuiamo una trasformazione del player_points

In [ ]:
# Converte i numeri in un altro formato
def transform_player_points(points):
    transformed = []
    for i in range(len(points)):
        points[i] = int(points[i])  # Conversione in int

        if i == 0:
            transformed.append(1 if points[i] > 0 else 0)  # Mantieniamo il primo valore
        else:
            transformed.append(1 if points[i] > points[i - 1] else 0)  # Confrontiamo i valori precedenti
    return transformed

# Convertiamo la stringa in una lista di interi
def safe_convert_to_list(points_str):
    try:
        # Usa ast.literal_eval per evitare problemi con la sintassi causato da []
        return ast.literal_eval(points_str)  # Converte la stringa in una lista
    except Exception as e:
        print(f"Errore nella conversione della stringa: {points_str}")
        return []

# Funzione di trasformazione che applica le funzioni
def transform_points_dataframe(row):
    points_list = safe_convert_to_list(row)
    return transform_player_points(points_list)

# Applica la funzione di trasformazione sul dataset senza usare lambda
dataset['player_points'] = dataset['player_points'].apply(transform_points_dataframe)

# Visualizza il dataframe
dataset



In [ ]:
sequence_input_dim = len(dataset['player_points'].iloc[0])  # La lunghezza della sequenza di player_points
sequence_length = max_length  # Lunghezza massima della sequenza
X_bin_input_dim = X_bin.shape[1]  # Numero di colonne in X_bin

# Definiamo gli input per il modello
sequence_input = Input(shape=(sequence_length,), dtype=tf.int32, name='sequence_input')  # Input sequenziale
bin_input = Input(shape=(X_bin_input_dim,), dtype=tf.float32, name='bin_input')  # Input variabili binarie

# Crea un layer di embedding per il sequenziale (opzionale, se le sequenze sono di tipo numerico, puoi ometterlo)
embedding_layer = Embedding(input_dim=100, output_dim=32, input_length=sequence_length)(sequence_input)  # Puoi variare i parametri

# Aggiungi un layer LSTM
lstm_layer = LSTM(64)(embedding_layer)  # Puoi regolare la dimensione dell'output dell'LSTM

# Aggiungi un layer Dropout per evitare overfitting
dropout_layer = Dropout(0.2)(lstm_layer)

# Combina l'output dell'LSTM con le variabili binarie
merged_layer = Concatenate()([dropout_layer, bin_input])

# Aggiungi un layer fully connected per la classificazione
dense_layer = Dense(64, activation='relu')(merged_layer)

# Aggiungi l'output finale
output_layer = Dense(1, activation='sigmoid')(dense_layer)  # 'sigmoid' per un problema di classificazione binaria

# Costruisci il modello
model = Model(inputs=[sequence_input, bin_input], outputs=output_layer)

# Compila il modello
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Visualizza la struttura del modello
model.summary()

In [ ]:
# Unione delle caratteristiche binarie e della sequenza
X_bin = np.array(dataset[['players_gender_M',	'players_gender_W',	'players_gender_X', 'is_final', 'final_set', 'opponent_final_set']].values)  # Concatenazione feature
X_seq = np.array(dataset['player_points'])

player_points = dataset['player_points']

# Padding delle sequenze per avere la stessa lunghezza
max_length = max(len(seq) for seq in player_points)  # Lunghezza massima della sequenza
padded_player_points = [seq + [-1] * (max_length - len(seq)) for seq in player_points]

dataset['player_points'] = padded_player_points

X_concate = [np.concatenate([x_bin, seq]) for x_bin, seq in zip(X_bin, X_seq)]

X = np.array(X_concate)

Y = np.array(dataset['set_result'].values)  # Target binari

In [ ]:
sequence_input = Input(shape=(max_length, 1), dtype='int32', name='sequence_input')  # 3D input shape
masked_sequence = Masking(mask_value=-1)(sequence_input)
lstm_out = LSTM(units=64)(masked_sequence)

# Define the input layer for binary features
binary_input = Input(shape=(X_bin.shape[1],), name='binary_input')
dense_bin = Dense(units=32, activation='relu')(binary_input)

# Concatenate the LSTM and Dense binary features output
concatenated = Concatenate()([lstm_out, dense_bin])

# Add a Dense layer for final classification
dense_out = Dense(units=32, activation='relu')(concatenated)
output = Dense(units=1, activation='sigmoid')(dense_out)

# Create the model
model = Model(inputs=[sequence_input, binary_input], outputs=output)

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the model with both inputs
#model.fit([padded_player_points, X_bin], Y, epochs=10, batch_size=64, validation_split=0.2)

In [ ]:
test_bin = np.array([[1, 0, 0, 0, 1, 0], [1, 0, 0, 0, 1, 0]])  # [players_gender, is_final, final_set, opponent_final_set]
test_seq = np.array([[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
                     [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]])  # Punti del giocatore


# Ora possiamo utilizzare questi dati per fare previsioni
predictions = model.predict([test_seq, test_bin])

# Stampiamo le previsioni
print("Predictions:")
print(predictions)

# Interpretazione delle previsioni
for i, pred in enumerate(predictions):
    print(f"Campione {i+1}: Probabilità di vittoria = {pred[0]:.2f}")